## Entendimento dos Dados (Data Understanding)

`Coleta inicial dos dados, familiarização com eles, identificação de problemas de qualidade, e geração de hipóteses iniciais.`


### Dados Abertos

Como fontes e dados possíveis, temos:

* Dados abertos da PGFN, disponíveis em <https://www.gov.br/pgfn/pt-br/assuntos/divida-ativa-da-uniao/transparencia-fiscal-1/dados-abertos>
* Dados abertos da Receita (base CNPJ), disponíveis em <https://dados.gov.br/dados/conjuntos-dados/cadastro-nacional-da-pessoa-juridica---cnpj>

#### PGFN

* **FGTS:** Refere-se a débitos de empregadores com o fundo de garantia.
* **PREV (Previdenciário):** Débitos de contribuições previdenciárias.
* **SIDA:** Conjunto de dados do sistema unificado da administração federal, com dados detalhados sobre devedores e sua situação perante a União.

Layout dos arquivos:

➜ FGTS PREV e SIDA
| Campo                     | Descrição                                                                 |
|---------------------------|---------------------------------------------------------------------------|
| CPF_CNPJ                  | CPF ou CNPJ do devedor inscrito na dívida ativa.                         |
| TIPO_PESSOA               | Indica se o devedor é pessoa física (F) ou jurídica (J).                 |
| TIPO_DEVEDOR              | Categoria do devedor: contribuinte individual, empresa, órgão público etc. |
| NOME_DEVEDOR              | Nome ou razão social do devedor.                                         |
| UF_DEVEDOR                | Unidade da Federação (estado) do devedor.                               |
| UNIDADE_RESPONSAVEL       | Unidade da PGFN responsável pela inscrição da dívida.                   |
| NUMERO_INSCRICAO          | Número único da inscrição da dívida ativa.                              |
| TIPO_SITUACAO_INSCRICAO   | Tipo da situação da inscrição: ativa, suspensa, extinta etc.            |
| SITUACAO_INSCRICAO        | Descrição detalhada da situação atual da dívida.                        |
| TIPO_CREDITO              | Tipo de crédito tributário ou não tributário (ex: FGTS, Multas).        |
| DATA_INSCRICAO            | Data da inscrição na dívida ativa.                                      |
| INDICADOR_AJUIZADO        | Indica se a dívida foi ajuizada: 'S' ou 'N'.                            |
| VALOR_CONSOLIDADO         | Valor total da dívida com acréscimos legais.                            |

➜ FGTS
| Campo                  | Descrição                                           |
|------------------------|---------------------------------------------------|
| ENTIDADE_RESPONSAVEL   | Entidade que gerou a inscrição do débito FGTS.    |
| UNIDADE_INSCRICAO      | Unidade/agência que realizou a inscrição do débito FGTS. |



#### CNPJ (Receita) 

**ideias:**
UF_DEVEDOR: tentar análise geográfica, indicando que alguns estados têm mais devedores. baixar dados do IBGE
UNIDADE_RESPONSAVEL: tentar analisar a carga de dívidas por unidade da PGFN.
SITUACAO_INSCRICAO: será que o status da dívida e como isso impacta a cobrança ou ajuizamento.
TIPO_CREDITO: tentar ver risco associado a diferentes tipos de crédito.

Tentar juntar com CNAE

### Exploratory

In [ ]:
import sys
import os

import networkx as nx
from collections import Counter
from networkx.drawing.nx_agraph import graphviz_layout

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from modules.utils.file import (
        find_zip_files_by_pattern,
        read_zip_csvs_as_df,
        save_parquet
)

from modules.utils.dataframe import (
    dataframe_info
)

from modules.utils.plot.style import ( 
    set_plot_style,
)

from modules.utils.plot.save import ( 
    save_figure
)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format
set_plot_style()
os.makedirs('reports/figures', exist_ok=True)

# data_root = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
data_root = '/home/nicolas/Documents/dados/pgfn/'


#### Load Data 

In [16]:
zip_files = find_zip_files_by_pattern(directory=f"{data_root}/dados/", pattern='FGTS.zip')
all_dfs = []

for f in zip_files:
    print(f"Processando arquivo: {f}")
    df = read_zip_csvs_as_df(f, encoding='latin1', verbose=False)
    all_dfs.append(df)

dadosDF = pd.concat(all_dfs, ignore_index=True)

Processando arquivo: /home/nicolas/Documents/dados/pgfn//dados/2020T2_Dados_abertos_FGTS.zip
Processando arquivo: /home/nicolas/Documents/dados/pgfn//dados/2020T1_Dados_abertos_FGTS.zip


In [ ]:
# apenas PJ
dadosDF = dadosDF[dadosDF['TIPO_PESSOA'].str.contains('jur', case=False, na=False)]

In [18]:
dadosDF['CPF_CNPJ'] = (
    dadosDF['CPF_CNPJ']
    .astype(str)
    .str.replace(r'\D', '', regex=True)
    .where(dadosDF['CPF_CNPJ'].notna())
)
dadosDF['DATA_INSCRICAO'] = pd.to_datetime(dadosDF['DATA_INSCRICAO'], format="%d/%m/%Y", errors='coerce')
dadosDF['ANO_INSCRICAO'] = dadosDF['DATA_INSCRICAO'].dt.year.astype('category')
dadosDF['MES_INSCRICAO'] = dadosDF['DATA_INSCRICAO'].dt.month.astype('category')
dadosDF['ANOMES_REFERENCIA'] = dadosDF['__source_file__'].str.extract(r'_(\d{6})\.csv$').astype('category')

dadosDF['TIPO_PESSOA'] = dadosDF['TIPO_PESSOA'].astype('category')
dadosDF['TIPO_DEVEDOR'] = dadosDF['TIPO_DEVEDOR'].astype('category')
dadosDF['UF_UNIDADE_RESPONSAVEL'] = dadosDF['UF_UNIDADE_RESPONSAVEL'].astype('category')
dadosDF['SITUACAO_INSCRICAO'] = dadosDF['SITUACAO_INSCRICAO'].astype('category')
dadosDF['UNIDADE_RESPONSAVEL'] = dadosDF['UNIDADE_RESPONSAVEL'].astype('category')
dadosDF['ENTIDADE_RESPONSAVEL'] = dadosDF['ENTIDADE_RESPONSAVEL'].astype('category')
dadosDF['UNIDADE_INSCRICAO'] = dadosDF['UNIDADE_INSCRICAO'].astype('category')
dadosDF['TIPO_SITUACAO_INSCRICAO'] = dadosDF['TIPO_SITUACAO_INSCRICAO'].astype('category')
dadosDF['INDICADOR_AJUIZADO'] = dadosDF['INDICADOR_AJUIZADO'].astype('category')
dadosDF['RECEITA_PRINCIPAL'] = dadosDF['RECEITA_PRINCIPAL'].astype('category')

dadosDF['VALOR_CONSOLIDADO'] = dadosDF['VALOR_CONSOLIDADO'].astype(float)

# dataframe_info(dadosDF)
dadosDF

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NOME_DEVEDOR,UF_UNIDADE_RESPONSAVEL,UNIDADE_RESPONSAVEL,ENTIDADE_RESPONSAVEL,UNIDADE_INSCRICAO,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,__source_file__,ANO_INSCRICAO,MES_INSCRICAO,ANOMES_REFERENCIA
0,02045981000119,Pessoa jurídica,Principal,SANCOL SANEAMENTO CONSTRUCAO E COMERCIO LTDA,AC,ACRE,PGFN,ACRE,FGAC201500072,Em cobrança,INSCRITA,Contribuições FGTS,2015-02-24,NAO,"3,177.31",arquivo_lai_FGTS_AC_202006.csv,2015,2,202006
1,02214258000116,Pessoa jurídica,Principal,I A DO MONTE,AC,ACRE,PGFN,ACRE,FGAC201100024,Em cobrança,AJUIZADA,Contribuições FGTS,2011-07-12,SIM,"4,751.29",arquivo_lai_FGTS_AC_202006.csv,2011,7,202006
2,09577695000143,Pessoa jurídica,Principal,EDNA NOBRE FERNANDES,AC,ACRE,PGFN,ACRE,FGAC201700422,Em cobrança,AJUIZADA,Contribuições FGTS,2017-12-27,SIM,"34,386.78",arquivo_lai_FGTS_AC_202006.csv,2017,12,202006
3,34706168000139,Pessoa jurídica,Principal,CONSTRUCAO PAVIMENTACAO E TRANSPORTES LTDA,AC,ACRE,PGFN,ACRE,FGAC201000046,Em cobrança,AJUIZADA,Contribuições FGTS,2010-09-30,SIM,"23,811.87",arquivo_lai_FGTS_AC_202006.csv,2010,9,202006
4,34706168000139,Pessoa jurídica,Principal,CONSTRUCAO PAVIMENTACAO E TRANSPORTES LTDA,AC,ACRE,PGFN,ACRE,FGAC201000057,Em cobrança,AJUIZADA,Contribuições FGTS,2010-11-18,SIM,"232,077.56",arquivo_lai_FGTS_AC_202006.csv,2010,11,202006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882193,07444090000159,Pessoa jurídica,Principal,GILMAR SOARES,TO,TOCANTINS,PGFN,TOCANTINS,FGTO200900147,Em cobrança,AJUIZADA,Contribuições FGTS,2009-08-04,SIM,"6,282.46",arquivo_lai_FGTS_TO_202002.csv,2009,8,202002
882194,10775040000169,Pessoa jurídica,Principal,COLEGIO POLIVALENTE DE PALMAS LTDA,TO,TOCANTINS,PGFN,TOCANTINS,FGTO201100077,Em cobrança,AJUIZADA,Contribuições FGTS,2011-08-18,SIM,"1,731.59",arquivo_lai_FGTS_TO_202002.csv,2011,8,202002
882195,10775040000169,Pessoa jurídica,Principal,COLEGIO POLIVALENTE DE PALMAS LTDA,TO,TOCANTINS,PGFN,TOCANTINS,CSTO201100078,Em cobrança,AJUIZADA,Contribuições LC110 (FGTS),2011-08-18,SIM,305.80,arquivo_lai_FGTS_TO_202002.csv,2011,8,202002
882196,02320284000129,Pessoa jurídica,Principal,R PEREIRA GOMES,TO,TOCANTINS,PGFN,TOCANTINS,FGTO200900185,Em cobrança,AJUIZADA,Contribuições FGTS,2009-10-27,SIM,"11,642.49",arquivo_lai_FGTS_TO_202002.csv,2009,10,202002


In [19]:
ignore = ['CPF_CNPJ','NOME_DEVEDOR','NUMERO_INSCRICAO','__source_file__']
dataframe_info(dadosDF, ignore=ignore)

Variáveis categóricas:
['TIPO_PESSOA', 'TIPO_DEVEDOR', 'UF_UNIDADE_RESPONSAVEL', 'UNIDADE_RESPONSAVEL', 'ENTIDADE_RESPONSAVEL', 'UNIDADE_INSCRICAO', 'TIPO_SITUACAO_INSCRICAO', 'SITUACAO_INSCRICAO', 'RECEITA_PRINCIPAL', 'INDICADOR_AJUIZADO', 'ANO_INSCRICAO', 'MES_INSCRICAO', 'ANOMES_REFERENCIA']

Número de categorias únicas:
TIPO_PESSOA                  1
TIPO_DEVEDOR                 1
UF_UNIDADE_RESPONSAVEL      27
UNIDADE_RESPONSAVEL        111
ENTIDADE_RESPONSAVEL         2
UNIDADE_INSCRICAO          111
TIPO_SITUACAO_INSCRICAO      4
SITUACAO_INSCRICAO          10
RECEITA_PRINCIPAL            2
INDICADOR_AJUIZADO           2
ANO_INSCRICAO               51
MES_INSCRICAO               12
ANOMES_REFERENCIA            2
dtype: int64

>>>>> TIPO_PESSOA <<<<<
Total de categorias: 1
TIPO_PESSOA
Pessoa jurídica    100.0%
Name: proportion, dtype: object

>>>>> TIPO_DEVEDOR <<<<<
Total de categorias: 1
TIPO_DEVEDOR
Principal    100.0%
Name: proportion, dtype: object

>>>>> UF_UNIDADE_RESPONSA

### Make file INSCRICAO_BASE

In [20]:
filepath = f"{data_root}/data/intermediate/inscricao_base.parquet"
columns = ['NUMERO_INSCRICAO', 'CPF_CNPJ', 'NOME_DEVEDOR', 'DATA_INSCRICAO', 'ANO_INSCRICAO', 'MES_INSCRICAO', 'TIPO_PESSOA', 'TIPO_DEVEDOR', 'UF_UNIDADE_RESPONSAVEL', 'UNIDADE_RESPONSAVEL', 'RECEITA_PRINCIPAL','ENTIDADE_RESPONSAVEL', 'UNIDADE_INSCRICAO']
df = save_parquet(df=dadosDF, path=filepath, columns=columns, unique=True)

File saved at: /home/nicolas/Documents/dados/pgfn//data/intermediate/inscricao_base.parquet


In [21]:
# df = pd.read_parquet(filepath)
df

,NUMERO_INSCRICAO,CPF_CNPJ,NOME_DEVEDOR,DATA_INSCRICAO,ANO_INSCRICAO,MES_INSCRICAO,TIPO_PESSOA,TIPO_DEVEDOR,UF_UNIDADE_RESPONSAVEL,UNIDADE_RESPONSAVEL,RECEITA_PRINCIPAL,ENTIDADE_RESPONSAVEL,UNIDADE_INSCRICAO
0,FGAC201500072,02045981000119,SANCOL SANEAMENTO CONSTRUCAO E COMERCIO LTDA,2015-02-24,2015,2,Pessoa jurídica,Principal,AC,ACRE,Contribuições FGTS,PGFN,ACRE
1,FGAC201100024,02214258000116,I A DO MONTE,2011-07-12,2011,7,Pessoa jurídica,Principal,AC,ACRE,Contribuições FGTS,PGFN,ACRE
2,FGAC201700422,09577695000143,EDNA NOBRE FERNANDES,2017-12-27,2017,12,Pessoa jurídica,Principal,AC,ACRE,Contribuições FGTS,PGFN,ACRE
3,FGAC201000046,34706168000139,CONSTRUCAO PAVIMENTACAO E TRANSPORTES LTDA,2010-09-30,2010,9,Pessoa jurídica,Principal,AC,ACRE,Contribuições FGTS,PGFN,ACRE
4,FGAC201000057,34706168000139,CONSTRUCAO PAVIMENTACAO E TRANSPORTES LTDA,2010-11-18,2010,11,Pessoa jurídica,Principal,AC,ACRE,Contribuições FGTS,PGFN,ACRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
881891,CSTO201800029,03601036000380,IPANEMA SEGURANCA LTDA,2018-06-28,2018,6,Pessoa jurídica,Principal,TO,TOCANTINS,Contribuições LC110 (FGTS),PGFN,TOCANTINS
881894,CSTO201900191,08665816000146,BUENO & CIA LTDA,2019-09-26,2019,9,Pessoa jurídica,Principal,TO,TOCANTINS,Contribuições LC110 (FGTS),PGFN,TOCANTINS
881940,FGTO201900074,07062767000194,CENTRO DE FORMACAO DE CONDUTORES DE VEICULOS A...,2019-06-19,2019,6,Pessoa jurídica,Principal,TO,TOCANTINS,Contribuições FGTS,PGFN,TOCANTINS
881958,FGTO200900187,09377997000410,MARGEN S.A.,2009-10-27,2009,10,Pessoa jurídica,Principal,TO,TOCANTINS,Contribuições FGTS,PGFN,TOCANTINS


### Make file INSCRICAO_ANOMES

In [22]:
filepath = f"{data_root}/data/intermediate/inscricao_anomes_valor.parquet"
columns = ['NUMERO_INSCRICAO', 'ANOMES_REFERENCIA', 'VALOR_CONSOLIDADO', 'TIPO_SITUACAO_INSCRICAO', 'SITUACAO_INSCRICAO', 'INDICADOR_AJUIZADO']
order_by = ['NUMERO_INSCRICAO','ANOMES_REFERENCIA']

df = save_parquet(df=dadosDF, path=filepath, columns=columns, order_by=order_by)
df

File saved at: /home/nicolas/Documents/dados/pgfn//data/intermediate/inscricao_anomes_valor.parquet


,NUMERO_INSCRICAO,ANOMES_REFERENCIA,VALOR_CONSOLIDADO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,INDICADOR_AJUIZADO
441016,CSAC200700028,202002,204.45,Em cobrança,AJUIZADA,SIM
374,CSAC200700028,202006,206.32,Em cobrança,AJUIZADA,SIM
441370,CSAC200700039,202002,331.56,Em cobrança,INSCRITA,NAO
339,CSAC200700039,202006,334.70,Em cobrança,INSCRITA,NAO
441123,CSAC200700043,202002,362.21,Em cobrança,INSCRITA,NAO
...,...,...,...,...,...,...
82467,FGUB000100060,202006,"19,895.60",Em cobrança,AJUIZADA,SIM
517604,FGUB000100069,202002,"105,398.99",Em cobrança,AJUIZADA,SIM
81129,FGUB000100069,202006,"106,122.17",Em cobrança,AJUIZADA,SIM
521562,FGUB000990267,202002,"7,657.65",Em cobrança,AJUIZADA,SIM


In [23]:
# Definir faixa 

### Make file INSCRICAO_SITUACAO

In [24]:
# Pivotea o dataframe para que cada coluna seja uma situação de inscrição
df = dadosDF.copy()
df['ANOMES_REFERENCIA'] = df['ANOMES_REFERENCIA'].astype(str)
df['SIT_COL'] = 'SIT_' + df['ANOMES_REFERENCIA']
# Faz o pivot (cada coluna SIT_YYYYMM será uma nova coluna, com valor SITUACAO_INSCRICAO)
df_pivot = df.pivot(index='NUMERO_INSCRICAO', 
                    columns='SIT_COL', 
                    values='SITUACAO_INSCRICAO')
df_pivot = df_pivot.reset_index()

# acrescenta a data de inscricao
df = dadosDF.copy()
df_inscricao = df[['NUMERO_INSCRICAO', 'DATA_INSCRICAO']].drop_duplicates()
duplicadas = df_inscricao[df_inscricao.duplicated(subset='NUMERO_INSCRICAO', keep=False)]
if not duplicadas.empty:
    print("Atenção! Existem inscrições com mais de uma data:")
    display(duplicadas)
df_completo = df_pivot.merge(df_inscricao, on='NUMERO_INSCRICAO', how='left')

# junta tudo
filepath = f"{data_root}/data/intermediate/inscricao_situacao.parquet"
order_by = ['DATA_INSCRICAO']
df = save_parquet(df=df_completo, path=filepath, order_by=order_by)
df

File saved at: /home/nicolas/Documents/dados/pgfn//data/intermediate/inscricao_situacao.parquet


,NUMERO_INSCRICAO,SIT_202002,SIT_202006,PRIMEIRO_ANOMES_AJUIZADO,DATA_INSCRICAO
91798,FGBU000100460,AJUIZADA,AJUIZADA,202002,1970-10-09
348914,FGSP000064051,AJUIZADA,AJUIZADA,202002,1970-10-15
109396,FGES000003013,AJUIZADA,AJUIZADA,202002,1970-10-27
243765,FGRJ000069409,AJUIZADA,AJUIZADA,202002,1970-11-30
249606,FGRJ000101273,AJUIZADA,AJUIZADA,202002,1970-12-04
...,...,...,...,...,...
432582,FGSP202001263,NaN,INSCRITA,NaN,2020-05-18
86279,FGBA202000203,NaN,INSCRITA,NaN,2020-05-19
337655,FGSC202000275,NaN,INSCR PARCELADA,NaN,2020-05-21
167664,FGMG202000618,NaN,INSCRITA,NaN,2020-05-29


Não da pra considerar primeiro ajuizamento pq ele pode sair e voltar da situação e para garantir ter toda

### Make file INSCRICAO_HIST_IND_AJUIZAMENTO

In [ ]:
# dadosDF['DATA_INSCRICAO'].hist(bins=5)
df = dadosDF.copy()
df.groupby('NUMERO_INSCRICAO')['INDICADOR_AJUIZADO'].shift()

In [42]:
# Cópia do dataframe original e preparação
df = dadosDF.copy()
df['ANOMES_REFERENCIA'] = df['ANOMES_REFERENCIA'].astype(int)
df.sort_values(by=['NUMERO_INSCRICAO', 'ANOMES_REFERENCIA'], inplace=True)

# Detecta mudanças de ajuizamento
df['AJUIZADO_SHIFT'] = df.groupby('NUMERO_INSCRICAO')['INDICADOR_AJUIZADO'].shift()
# df['MUDANCA_AJUIZADO'] = (
#     (df['INDICADOR_AJUIZADO'] != df['AJUIZADO_SHIFT']) & 
#     (df['AJUIZADO_SHIFT'].notna())
# )
df['MUDANCA_AJUIZADO_N2S'] = (df['INDICADOR_AJUIZADO'] == 'SIM') & (df['AJUIZADO_SHIFT'] == 'NAO')
df['MUDANCA_AJUIZADO_S2N'] = (df['INDICADOR_AJUIZADO'] == 'NAO') & (df['AJUIZADO_SHIFT'] == 'SIM')

# Cria colunas SIT_YYYYMM e MUDOU_YYYYMM
df['ANOMES_REFERENCIA'] = df['ANOMES_REFERENCIA'].astype(str)
df['SIT_COL'] = 'IND_AJUIZ_' + df['ANOMES_REFERENCIA']
df['MUDOU_COL_S2N']  = 'MUDOU_S2N_' + df['ANOMES_REFERENCIA']
df['MUDOU_COL_N2S'] = 'MUDOU_N2S_' + df['ANOMES_REFERENCIA']

# Pivot das situações
df_pivot = df.pivot(index='NUMERO_INSCRICAO', columns='SIT_COL', values='INDICADOR_AJUIZADO').reset_index()

# Pivot das mudanças
df_mudouS2N = df.pivot(index='NUMERO_INSCRICAO', columns='MUDOU_COL_S2N', values='MUDANCA_AJUIZADO_N2S').reset_index()

df_mudouN2S = df.pivot(index='NUMERO_INSCRICAO', columns='MUDOU_COL_N2S', values='MUDANCA_AJUIZADO_S2N').reset_index()

# Junta os dois pivot
df_pivot = df_pivot.merge(df_mudouS2N, on='NUMERO_INSCRICAO', how='outer') \
    .merge(df_mudouN2S, on='NUMERO_INSCRICAO', how='outer')
df_pivot

,NUMERO_INSCRICAO,IND_AJUIZ_202002,IND_AJUIZ_202006,MUDOU_S2N_202002,MUDOU_S2N_202006,MUDOU_N2S_202002,MUDOU_N2S_202006
0,CSAC200700028,SIM,SIM,False,False,False,False
1,CSAC200700039,NAO,NAO,False,False,False,False
2,CSAC200700043,NAO,NAO,False,False,False,False
3,CSAC200700046,SIM,SIM,False,False,False,False
4,CSAC200700048,SIM,SIM,False,False,False,False
...,...,...,...,...,...,...,...
442741,FGUB000099128,SIM,SIM,False,False,False,False
442742,FGUB000099159,SIM,SIM,False,False,False,False
442743,FGUB000100060,SIM,SIM,False,False,False,False
442744,FGUB000100069,SIM,SIM,False,False,False,False


Tem inscricao que estavam como SIM e voltaram para NAO: FGGO201000004